In [ ]:
import Logging: global_logger
import TerminalLoggers: TerminalLogger
global_logger(TerminalLogger())

In [ ]:
using SparseArrays

In [ ]:
struct FDMesh2D
    xl::Float64
    xr::Float64
    yl::Float64
    yr::Float64
    dx::Float64
    dy::Float64
    nx::Int64
    ny::Int64
    # nodes::Vector{Tuple{Float64, Float64}}
    nodes_idxs::CartesianIndices{2}
    midpoints_idxs_x::CartesianIndices{2} # On horizontal edges, staggered on x axis
    midpoints_idxs_y::CartesianIndices{2} # On vertical edges, staggered on y axis
end

In [ ]:
function FDMesh2D(xl::Float64, xr::Float64, nx::Int64, yl::Float64, yr::Float64, ny::Int64)
    hx = (xr - xl) / nx
    hy = (yr - yl) / ny
    metrics = [hx, hy]
    nodes_idxs = CartesianIndices((nx + 1, ny + 1))
    midpoints_idxs_x = CartesianIndices((nx, ny + 1))
    midpoints_idxs_y = CartesianIndices((nx + 1, ny))
    return FDMesh2D(xl, xr, yl, yr, hx, hy, nx, ny, nodes_idxs, midpoints_idxs_x, midpoints_idxs_y)
    # nodes = [(collect(Tuple(idx)).-1) .* metrics for idx in nodes_idxs] |> vec
end  

In [ ]:
function gridPointToLeftEdgeNeighbor(mesh::FDMesh2D, x::Int, y::Int)
    mesh.nx * (y - 1) + (x - 1)
end

function gridPointToRightEdgeNeighbor(mesh::FDMesh2D, x::Int, y::Int)
    mesh.nx * (y - 1) + x
end

function gridPointToTopEdgeNeighbor(mesh::FDMesh2D, x::Int, y::Int)
    (mesh.nx + 1) * (y - 1) + x 
end

function gridPointToBottomEdgeNeighbor(mesh::FDMesh2D, x::Int, y::Int)
    (mesh.nx + 1) * (y - 2) + x 
end

In [ ]:
function generateLinearEtaMatrix(mesh::FDMesh2D)
    eta_start = 0
    u_start = eta_start + (mesh.nx + 1) * (mesh.ny + 1)
    v_start = u_start + (mesh.nx) * (mesh.ny + 1)
    A = spzeros(Float64, (mesh.nx + 1) * (mesh.ny + 1), (mesh.nx + 1) * (mesh.ny + 1) + (mesh.nx) * (mesh.ny + 1) + (mesh.nx + 1) * (mesh.ny))

    
    i = 0
    for idx in mesh.nodes_idxs
        i += 1    
        x, y = Tuple(idx)      
        if x == 1 && y == 1
            # Bottom left corner
            continue
            
        elseif x == 1 && y == (mesh.ny + 1)
            # Top left corner
            continue
            
        elseif x == (mesh.nx + 1) && y == (mesh.ny + 1)
            # Top right corner
            left_midpoint = gridPointToLeftEdgeNeighbor(mesh, x, y)
            bottom_midpoint = gridPointToBottomEdgeNeighbor(mesh, x, y)
            
            A[i,u_start + left_midpoint] = -1/mesh.dx 
            A[i,v_start + bottom_midpoint] = -1/mesh.dy
            
        elseif x == (mesh.nx + 1) && y == 1 
            # Bottom right corner
            left_midpoint = gridPointToLeftEdgeNeighbor(mesh, x, y)
            top_midpoint = gridPointToTopEdgeNeighbor(mesh, x, y)
            
            A[i,u_start + left_midpoint] = -1/mesh.dx 
            A[i,v_start + top_midpoint] = 1/mesh.dy
            
        elseif x == 1
            # Left boundary
            continue
            
        elseif y == 1
            # Bottom boundary
            left_midpoint = gridPointToLeftEdgeNeighbor(mesh, x, y)
            right_midpoint = gridPointToRightEdgeNeighbor(mesh, x, y)
            top_midpoint = gridPointToTopEdgeNeighbor(mesh, x, y)
            
            A[i,u_start + left_midpoint] = -1/mesh.dx 
            A[i,u_start + right_midpoint] = 1/mesh.dx 
            A[i,v_start + top_midpoint] = 1/mesh.dy
            
        elseif y == (mesh.ny + 1)
            # Top boundary
            left_midpoint = gridPointToLeftEdgeNeighbor(mesh, x, y)
            right_midpoint = gridPointToRightEdgeNeighbor(mesh, x, y)
            bottom_midpoint = gridPointToBottomEdgeNeighbor(mesh, x, y)
            
            A[i,u_start + left_midpoint] = -1/mesh.dx 
            A[i,u_start + right_midpoint] = 1/mesh.dx 
            A[i,v_start + bottom_midpoint] = -1/mesh.dy
            
        elseif x == (mesh.nx + 1)
            # Right boundary
            left_midpoint = gridPointToLeftEdgeNeighbor(mesh, x, y)
            top_midpoint = gridPointToTopEdgeNeighbor(mesh, x, y)
            bottom_midpoint = gridPointToBottomEdgeNeighbor(mesh, x, y)
            
            A[i,u_start + left_midpoint] = -1/mesh.dx 
            A[i,v_start + top_midpoint] = 1/mesh.dy
            A[i,v_start + bottom_midpoint] = -1/mesh.dy
            
        else
            # Inside
            left_midpoint = gridPointToLeftEdgeNeighbor(mesh, x, y)
            right_midpoint = gridPointToRightEdgeNeighbor(mesh, x, y)
            top_midpoint = gridPointToTopEdgeNeighbor(mesh, x, y)
            bottom_midpoint = gridPointToBottomEdgeNeighbor(mesh, x, y)
            
            A[i,u_start + left_midpoint] = -1/mesh.dx 
            A[i,u_start + right_midpoint] = 1/mesh.dx 
            A[i,v_start + top_midpoint] = 1/mesh.dy
            A[i,v_start + bottom_midpoint] = -1/mesh.dy
        end
            
    end
    A
end

In [ ]:
function horizontalEdgePointLeftGridPointNeighbor(mesh::FDMesh2D, x::Int, y::Int)
    (mesh.nx + 1) * (y - 1) + x
end

function horizontalEdgePointRightGridPointNeighbor(mesh::FDMesh2D, x::Int, y::Int)
    (mesh.nx + 1) * (y - 1) + (x + 1)
end

In [ ]:
function generateLinearUMatrix(mesh::FDMesh2D)
    eta_start = 0
    u_start = eta_start + (mesh.nx + 1) * (mesh.ny + 1)
    v_start = u_start + (mesh.nx) * (mesh.ny + 1)
    A = spzeros(Float64, (mesh.nx) * (mesh.ny + 1), (mesh.nx + 1) * (mesh.ny + 1) + (mesh.nx) * (mesh.ny + 1) + (mesh.nx + 1) * (mesh.ny))

    i = 0
    for idx in mesh.midpoints_idxs_x
        i += 1    
        x, y = Tuple(idx)
        left_grid_point = horizontalEdgePointLeftGridPointNeighbor(mesh, x, y)
        right_grid_point = horizontalEdgePointRightGridPointNeighbor(mesh, x, y)
        A[i, eta_start + left_grid_point] = -1/mesh.dx
        A[i, eta_start + right_grid_point] = 1/mesh.dx
    end
    A
end

In [ ]:
function verticalEdgePointTopGridPointNeighbor(mesh::FDMesh2D, x::Int, y::Int)
    (mesh.nx + 1) * y + x
end

function verticalEdgePointBottomGridPointNeighbor(mesh::FDMesh2D, x::Int, y::Int)
    (mesh.nx + 1) * (y - 1) + x
end

In [ ]:
function generateLinearVMatrix(mesh::FDMesh2D)
    eta_start = 0
    u_start = eta_start + (mesh.nx + 1) * (mesh.ny + 1)
    v_start = u_start + (mesh.nx) * (mesh.ny + 1)
    A = spzeros(Float64, (mesh.nx + 1) * (mesh.ny), (mesh.nx + 1) * (mesh.ny + 1) + (mesh.nx) * (mesh.ny + 1) + (mesh.nx + 1) * (mesh.ny))

    i = 0
    for idx in mesh.midpoints_idxs_y
        i += 1    
        x, y = Tuple(idx)
        top_grid_point = verticalEdgePointTopGridPointNeighbor(mesh, x, y)
        bottom_grid_point = verticalEdgePointBottomGridPointNeighbor(mesh, x, y)
        A[i, eta_start + top_grid_point] = 1/mesh.dx
        A[i, eta_start + bottom_grid_point] = -1/mesh.dx
    end
    A
end

In [ ]:
function generateLinearFullMatrix(mesh::FDMesh2D, H::Float64, g::Float64)
    eta_mat =  - H .* generateLinearEtaMatrix(mesh)
    u_mat = - g .* generateLinearUMatrix(mesh)
    v_mat = - g .* generateLinearVMatrix(mesh)
    [eta_mat; u_mat; v_mat]
end

In [ ]:
function generateConvectiveIdentityMatrix(mesh::FDMesh2D)
    eta_start = 0
    u_start = eta_start + (mesh.nx + 1) * (mesh.ny + 1)
    v_start = u_start + (mesh.nx) * (mesh.ny + 1)

    eta_size = (mesh.nx + 1) * (mesh.ny + 1)
    u_size = (mesh.nx) * (mesh.ny + 1)
    v_size = (mesh.nx + 1) * (mesh.ny)
    
    empty_top = spzeros(Float64, eta_size, eta_size + u_size + v_size)
    middle = [
        spzeros(Float64, u_size, eta_size) spdiagm(0 => ones(u_size)) spzeros(Float64, u_size, v_size)
    ]
    bottom = [
        spzeros(Float64, v_size, eta_size) spzeros(Float64, v_size, u_size) spdiagm(0 => ones(v_size))
    ]

    [empty_top; middle; bottom]
end

In [ ]:
function horizontalEdgePointLeftEdgeNeighbor(mesh::FDMesh2D, x::Int, y::Int)
    (mesh.nx) * (y - 1) + (x - 1)
end

function horizontalEdgePointRightEdgeNeighbor(mesh::FDMesh2D, x::Int, y::Int)
    (mesh.nx) * (y - 1) + (x + 1)
end

function verticalEdgePointTopEdgeNeighbor(mesh::FDMesh2D, x::Int, y::Int)
    (mesh.nx + 1) * (y) + (x)
end

function verticalEdgePointBottomEdgeNeighbor(mesh::FDMesh2D, x::Int, y::Int)
    (mesh.nx + 1) * (y - 2) + (x)
end

In [ ]:
function generateConvectiveMatrix(mesh::FDMesh2D)
    eta_start = 0
    u_start = eta_start + (mesh.nx + 1) * (mesh.ny + 1)
    v_start = u_start + (mesh.nx) * (mesh.ny + 1)

    eta_size = (mesh.nx + 1) * (mesh.ny + 1)
    u_size = (mesh.nx) * (mesh.ny + 1)
    v_size = (mesh.nx + 1) * (mesh.ny)
    
    empty_top = spzeros(Float64, eta_size, eta_size + u_size + v_size)
    middle = spzeros(Float64, u_size, eta_size + u_size + v_size)
    bottom = spzeros(Float64, v_size, eta_size + u_size + v_size)

    i = 0
    for idx in mesh.midpoints_idxs_x
        i += 1    
        x, y = Tuple(idx)
        if x == 1
            # Left boundary
            # right_edge_neighbor = horizontalEdgePointRightEdgeNeighbor(mesh, x, y)
            # righter_edge_neighbor = horizontalEdgePointRightEdgeNeighbor(mesh, x+1, y)
            # middle[i, u_start + right_edge_neighbor] = 4 / (2 * mesh.dx)
            # middle[i, u_start + righter_edge_neighbor] = -1 / (2 * mesh.dx)
            # middle[i, u_start + i] = -3 / (2 * mesh.dx)
            right_edge_neighbor = horizontalEdgePointRightEdgeNeighbor(mesh, x, y)
            middle[i, u_start + right_edge_neighbor] = 1 / (2 * mesh.dx)
        elseif x == mesh.nx
            # Right boundary
            left_edge_neighbor = horizontalEdgePointLeftEdgeNeighbor(mesh, x, y)
            middle[i, u_start + left_edge_neighbor] = -1 / (2 * mesh.dx)
        else
            # Middle
            left_edge_neighbor = horizontalEdgePointLeftEdgeNeighbor(mesh, x, y)
            right_edge_neighbor = horizontalEdgePointRightEdgeNeighbor(mesh, x, y)
            middle[i, u_start + left_edge_neighbor] = -1 / (2 * mesh.dx)
            middle[i, u_start + right_edge_neighbor] = 1 / (2 * mesh.dx)
        end
    end
    
    i = 0
    for idx in mesh.midpoints_idxs_y
        i += 1    
        x, y = Tuple(idx)
        if y == 1
            # Bottom boundary
            top_edge_neighbor = verticalEdgePointTopEdgeNeighbor(mesh, x, y)
            bottom[i, v_start + top_edge_neighbor] = 1 / (2 * mesh.dy)
        elseif y == mesh.ny
            # Top boundary
            bottom_edge_neighbor = verticalEdgePointBottomEdgeNeighbor(mesh, x, y)
            bottom[i, v_start + bottom_edge_neighbor] = -1 / (2 * mesh.dy)
        else
            # Middle
            top_edge_neighbor = verticalEdgePointTopEdgeNeighbor(mesh, x, y)
            bottom_edge_neighbor = verticalEdgePointBottomEdgeNeighbor(mesh, x, y)
            bottom[i, v_start + top_edge_neighbor] = 1 / (2 * mesh.dy)
            bottom[i, v_start + bottom_edge_neighbor] = -1 / (2 * mesh.dy)
        end
    end

    [empty_top; middle; bottom]
end

In [ ]:
function horizontalEdgePointTopRightVerticalNeighbor(mesh::FDMesh2D, x::Int, y::Int)
    (mesh.nx + 1) * (y - 1) + (x + 1)
end

function horizontalEdgePointTopLeftVerticalNeighbor(mesh::FDMesh2D, x::Int, y::Int)
    (mesh.nx + 1) * (y - 1) + (x)
end

function horizontalEdgePointBottomRightVerticalNeighbor(mesh::FDMesh2D, x::Int, y::Int)
    (mesh.nx + 1) * (y - 2) + (x + 1)
end

function horizontalEdgePointBottomLeftVerticalNeighbor(mesh::FDMesh2D, x::Int, y::Int)
    (mesh.nx + 1) * (y - 2) + (x)
end

function verticalEdgePointTopRightHorizontalNeighbor(mesh::FDMesh2D, x::Int, y::Int)
    (mesh.nx) * (y) + (x)
end

function verticalEdgePointTopLeftHorizontalNeighbor(mesh::FDMesh2D, x::Int, y::Int)
    (mesh.nx) * (y) + (x - 1)
end

function verticalEdgePointBottomRightHorizontalNeighbor(mesh::FDMesh2D, x::Int, y::Int)
    (mesh.nx) * (y - 1) + (x)
end

function verticalEdgePointBottomLeftHorizontalNeighbor(mesh::FDMesh2D, x::Int, y::Int)
    (mesh.nx) * (y - 1) + (x - 1)
end

In [ ]:
function generateConvectiveAvgMatrix(mesh::FDMesh2D)
    eta_start = 0
    u_start = eta_start + (mesh.nx + 1) * (mesh.ny + 1)
    v_start = u_start + (mesh.nx) * (mesh.ny + 1)

    eta_size = (mesh.nx + 1) * (mesh.ny + 1)
    u_size = (mesh.nx) * (mesh.ny + 1)
    v_size = (mesh.nx + 1) * (mesh.ny)
    
    empty_top = spzeros(Float64, eta_size, eta_size + u_size + v_size)

    middle = spzeros(Float64, u_size, eta_size + u_size + v_size)
    i = 0
    for idx in mesh.midpoints_idxs_x
        i += 1    
        x, y = Tuple(idx)
        if y == 1
            # Bottom boundary
            top_right_vertical_neighbor = horizontalEdgePointTopRightVerticalNeighbor(mesh, x, y)
            top_left_vertical_neighbor = horizontalEdgePointTopLeftVerticalNeighbor(mesh, x, y)
            middle[i, v_start + top_right_vertical_neighbor] = 0.25
            middle[i, v_start + top_left_vertical_neighbor] = 0.25
        elseif y == mesh.ny + 1
            # Top boundary
            bottom_right_vertical_neighbor = horizontalEdgePointBottomRightVerticalNeighbor(mesh, x, y)
            bottom_left_vertical_neighbor = horizontalEdgePointBottomLeftVerticalNeighbor(mesh, x, y)
            middle[i, v_start + bottom_right_vertical_neighbor] = 0.25
            middle[i, v_start + bottom_left_vertical_neighbor] = 0.25
        else
            # Middle
            top_right_vertical_neighbor = horizontalEdgePointTopRightVerticalNeighbor(mesh, x, y)
            top_left_vertical_neighbor = horizontalEdgePointTopLeftVerticalNeighbor(mesh, x, y)
            bottom_right_vertical_neighbor = horizontalEdgePointBottomRightVerticalNeighbor(mesh, x, y)
            bottom_left_vertical_neighbor = horizontalEdgePointBottomLeftVerticalNeighbor(mesh, x, y)
            middle[i, v_start + top_right_vertical_neighbor] = 0.25
            middle[i, v_start + top_left_vertical_neighbor] = 0.25
            middle[i, v_start + bottom_right_vertical_neighbor] = 0.25
            middle[i, v_start + bottom_left_vertical_neighbor] = 0.25
        end
    end

    bottom = spzeros(Float64, v_size, eta_size + u_size + v_size)
    i = 0
    for idx in mesh.midpoints_idxs_y
        i += 1    
        x, y = Tuple(idx)
        if x == 1
            # Left boundary
            top_right_vertical_neighbor = horizontalEdgePointTopRightVerticalNeighbor(mesh, x, y)
            bottom_right_vertical_neighbor = horizontalEdgePointBottomRightVerticalNeighbor(mesh, x, y)
            bottom[i, u_start + top_right_vertical_neighbor] = 0.25
            bottom[i, u_start + bottom_right_vertical_neighbor] = 0.25
        elseif x == mesh.nx + 1
            # Right boundary
            top_left_vertical_neighbor = horizontalEdgePointTopLeftVerticalNeighbor(mesh, x, y)
            bottom_left_vertical_neighbor = horizontalEdgePointBottomLeftVerticalNeighbor(mesh, x, y)
            bottom[i, u_start + top_left_vertical_neighbor] = 0.25
            bottom[i, u_start + bottom_left_vertical_neighbor] = 0.25
        else
            # Middle
            top_right_vertical_neighbor = horizontalEdgePointTopRightVerticalNeighbor(mesh, x, y)
            top_left_vertical_neighbor = horizontalEdgePointTopLeftVerticalNeighbor(mesh, x, y)
            bottom_right_vertical_neighbor = horizontalEdgePointBottomRightVerticalNeighbor(mesh, x, y)
            bottom_left_vertical_neighbor = horizontalEdgePointBottomLeftVerticalNeighbor(mesh, x, y)
            bottom[i, u_start + top_right_vertical_neighbor] = 0.25
            bottom[i, u_start + top_left_vertical_neighbor] = 0.25
            bottom[i, u_start + bottom_right_vertical_neighbor] = 0.25
            bottom[i, u_start + bottom_left_vertical_neighbor] = 0.25
        end
    end

    [empty_top; middle; bottom]
end

In [ ]:
function horizontalEdgePointTopHorizontalNeighbor(mesh::FDMesh2D, x::Int, y::Int)
    (mesh.nx) * (y) + (x)
end

function horizontalEdgePointBottomHorizontalNeighbor(mesh::FDMesh2D, x::Int, y::Int)
    (mesh.nx) * (y - 2) + (x)
end

function verticalEdgePointLeftVerticalNeighbor(mesh::FDMesh2D, x::Int, y::Int)
    (mesh.nx + 1) * (y - 1) + (x - 1)
end

function verticalEdgePointRightVerticalNeighbor(mesh::FDMesh2D, x::Int, y::Int)
    (mesh.nx + 1) * (y - 1) + (x + 1)
end

In [ ]:
function generatePerpendicularConvectiveMatrix(mesh::FDMesh2D)
    eta_start = 0
    u_start = eta_start + (mesh.nx + 1) * (mesh.ny + 1)
    v_start = u_start + (mesh.nx) * (mesh.ny + 1)

    eta_size = (mesh.nx + 1) * (mesh.ny + 1)
    u_size = (mesh.nx) * (mesh.ny + 1)
    v_size = (mesh.nx + 1) * (mesh.ny)
    
    empty_top = spzeros(Float64, eta_size, eta_size + u_size + v_size)

    middle = spzeros(Float64, u_size, eta_size + u_size + v_size)
    i = 0
    for idx in mesh.midpoints_idxs_x
        i += 1    
        x, y = Tuple(idx)
        if y == 1
            # Bottom boundary
        elseif y == mesh.ny + 1
            # Top boundary
        else
            # Middle
            top_horizontal_neighbor = horizontalEdgePointTopHorizontalNeighbor(mesh, x, y)
            bottom_horizontal_neighbor = horizontalEdgePointBottomHorizontalNeighbor(mesh, x, y)
            middle[i, u_start + top_horizontal_neighbor] = 1 / (2 * mesh.dy)
            middle[i, u_start + bottom_horizontal_neighbor] = -1 / (2 * mesh.dy)
        end
    end

    bottom = spzeros(Float64, v_size, eta_size + u_size + v_size)
    i = 0
    for idx in mesh.midpoints_idxs_y
        i += 1    
        x, y = Tuple(idx)
        if x == 1
            # Left boundary
            # right_vertical_neighbor = verticalEdgePointRightVerticalNeighbor(mesh, x, y)
            # righter_vertical_neighbor = verticalEdgePointRightVerticalNeighbor(mesh, x+1, y)
            # bottom[i, v_start + right_vertical_neighbor] = 4 / (2 * mesh.dx)
            # bottom[i, v_start + righter_vertical_neighbor] = -1 / (2 * mesh.dx)
            # bottom[i, v_start + i] = -3 / (2 * mesh.dx)
            right_vertical_neighbor = verticalEdgePointRightVerticalNeighbor(mesh, x, y)
            bottom[i, v_start + right_vertical_neighbor] = 1 / (2 * mesh.dx)
        elseif x == mesh.nx + 1
            # Right boundary
            left_vertical_neighbor = verticalEdgePointLeftVerticalNeighbor(mesh, x, y)
            bottom[i, v_start + left_vertical_neighbor] = -1 / (2 * mesh.dx)
        else
            # Middle
            left_vertical_neighbor = verticalEdgePointLeftVerticalNeighbor(mesh, x, y)
            right_vertical_neighbor = verticalEdgePointRightVerticalNeighbor(mesh, x, y)
            bottom[i, v_start + left_vertical_neighbor] = -1 / (2 * mesh.dx)
            bottom[i, v_start + right_vertical_neighbor] = 1 / (2 * mesh.dx)
        end
    end

    [empty_top; middle; bottom]
end

In [ ]:
function f1!(dw, w, p, t)
    mesh, A, B, C, D, E, ω, F_zero, H, Cd = p
    i = 0
    for idx in mesh.nodes_idxs
        i += 1
        x, y = Tuple(idx) 
        if x == 1 && y <= (2 * mesh.ny / 3) && y >= (mesh.ny / 3)
            dw[i] = 0
            w[i] = F_zero * sin(ω * t) 
        end
    end
    dw .= (A * w)
end

function f2!(dw, w, p, t)
    mesh, A, B, C, D, E, ω, F_zero, H, Cd = p
    i = 0
    for idx in mesh.nodes_idxs
        i += 1
        x, y = Tuple(idx) 
        if x == 1 && y <= (2 * mesh.ny / 3) && y >= (mesh.ny / 3)
            dw[i] = 0
            w[i] = F_zero * sin(ω * t) 
        end
    end
    dw .= ((B * w) .* ((-C) * w)) .+ ((D * w) .* ((-E) * w)) .- ((Cd/H) * ((B * w) .* (((B * w) .* (B * w)) .+ ((D * w) .* (D * w)))))
end

In [ ]:
xl, xr = 0.0, 20.0
yl, yr = 0.0, 20.0
nx, ny = 75, 75
mesh = FDMesh2D(xl ,xr, nx, yl, yr, ny)

H = 2.0
g = 9.81

In [ ]:
A = generateLinearFullMatrix(mesh, H, g)

In [ ]:
B = generateConvectiveIdentityMatrix(mesh)

In [ ]:
C = generateConvectiveMatrix(mesh)

In [ ]:
D = generateConvectiveAvgMatrix(mesh)

In [ ]:
E = generatePerpendicularConvectiveMatrix(mesh)

In [ ]:
using DifferentialEquations

In [ ]:
using Plots

In [ ]:
ω = 1.0 * pi
F_zero = 0.5
Cd = 0.5

p = (mesh, A, B, C, D, E, ω, F_zero, H, Cd)

u0 = [0.0 for xi = 1:((mesh.nx + 1) * (mesh.ny + 1) + (mesh.nx) * (mesh.ny + 1) + (mesh.nx + 1) * (mesh.ny))]

tspan = (0.0, 30.0)

prob = SplitODEProblem(f1!, f2!, u0, tspan, p)
sol = @time solve(prob, reltol = 1e-8, abstol = 1e-8, save_everystep = true, progress=true, progress_steps=50)

In [ ]:
t_start, t_end = tspan
framerate = 30
dt = 1 / framerate

total = 0.0
j = 1
indices::Vector{Int64} = []

for i in 1:(t_end-t_start)*framerate
    target = i * dt
    while total < target && j < length(sol.t)
        total = sol.t[j]
        j+=1
    end
    push!(indices, j)
end

In [ ]:
anim = @animate for i in 1:(t_end-t_start)*framerate
    eta = sol.u[indices[Int64(i)]][1:((mesh.nx+1)*(mesh.ny+1))]
    eta_mat = reshape(eta, (mesh.nx+1), (mesh.ny+1))
    eta_mat = transpose(eta_mat)
    eta_mat = reverse(eta_mat, dims=1)

    u = sol.u[indices[Int64(i)]][((mesh.nx+1)*(mesh.ny+1))+1:((mesh.nx+1)*(mesh.ny+1))+((mesh.nx)*(mesh.ny+1))]
    u_mat = reshape(u, (mesh.nx), (mesh.ny+1))
    u_mat = transpose(u_mat)
    u_mat = reverse(u_mat, dims=1)
    
    v = sol.u[indices[Int64(i)]][((mesh.nx+1)*(mesh.ny+1))+((mesh.nx)*(mesh.ny+1))+1:((mesh.nx+1)*(mesh.ny+1))+((mesh.nx)*(mesh.ny+1))+((mesh.nx+1)*(mesh.ny))]
    v_mat = reshape(v, (mesh.nx+1), (mesh.ny))
    v_mat = transpose(v_mat)
    v_mat = reverse(v_mat, dims=1)
    
    eta_heatmap = heatmap(eta_mat, clims=(-2.0, 2.0), title=string("t = ", round(i *dt, digits=2)))
    u_heatmap = heatmap(u_mat, clims=(-2.0, 2.0))
    v_heatmap = heatmap(v_mat, clims=(-2.0, 2.0))

    plot(eta_heatmap, u_heatmap, v_heatmap; layout=(3, 1), size=(400, 1000))
end

gif(anim, "SWE_2D.gif", fps=framerate)